In [1]:
import pysam
from pysam import VariantFile as vcf
import operator
from math import log2
import pandas as pd
from pandas import DataFrame as dataframe
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import pdist, squareform
import scipy
import  os
import os.path
import matplotlib.colors as mcolors
from scipy import stats

In [4]:
#/data2/wangxuedong/mhc_test_data/first_review/race/split_chr1_xac.csv
def integrate_csv_chromosome(filespath):
    dic_file=dict()
    for i in range(22):
        dic_file["chr"+str(i+1)]=[]
    #print(dic_file)
    for filepath in filespath:
        if filepath.split("/")[-1]=="mhc.csv":
            dic_file["chr6"].append(filepath)
        else:
            dic_file[filepath.split("/")[-1].split(".")[0].split("_")[1]].append(filepath)
    #print(dic_file)
    for chr_,file_paths in dic_file.items():
        merged_data = pd.DataFrame()
        for file in file_paths:
            data = pd.read_csv(file)
            merged_data = merged_data.append(data, ignore_index=True)
        merged_data_sorted = merged_data.sort_values(by=merged_data.columns[0])
        merged_folderpath="/data2/wangxuedong/mhc_test_data/first_review/similar_region_race_integration/"+chr_+".csv"
        merged_data_sorted = merged_data_sorted.drop_duplicates(subset=merged_data_sorted.columns[0])
        if chr_=="chr6":
            merged_data_ls=merged_data_sorted[merged_data_sorted[merged_data_sorted.columns[0]] < 29720403]
            merged_data_ls.to_csv("/data2/wangxuedong/mhc_test_data/first_review/similar_region_race_integration/chr6_partA.csv",index=False)
            merged_data_mid=merged_data_sorted[(merged_data_sorted[merged_data_sorted.columns[0]] >= 29720403) & (merged_data_sorted[merged_data_sorted.columns[0]] <= 33129982)]
            merged_data_mid.to_csv("/data2/wangxuedong/mhc_test_data/first_review/similar_region_race_integration/chr6_mhcpart.csv",index=False)
            merged_data_bt=merged_data_sorted[merged_data_sorted[merged_data_sorted.columns[0]] > 33129982 ]
            merged_data_bt.to_csv("/data2/wangxuedong/mhc_test_data/first_review/similar_region_race_integration/chr6_partC.csv",index=False)           
        else:
            merged_data_sorted.to_csv(merged_folderpath, index=False)
      
            
need_merge_filespath="/data2/wangxuedong/mhc_test_data/first_review/similar_region_race/"
filespath=[need_merge_filespath+file for file in os.listdir(need_merge_filespath)]
# filespath
integrate_csv_chromosome(filespath)
        

/tmp/ipykernel_3838960/3678907616.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_data = merged_data.append(data, ignore_index=True)
/tmp/ipykernel_3838960/3678907616.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_data = merged_data.append(data, ignore_index=True)
/tmp/ipykernel_3838960/3678907616.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_data = merged_data.append(data, ignore_index=True)
/tmp/ipykernel_3838960/3678907616.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_data = merged_data.append(data, ignore_index=True)
/tmp/ipykernel_3838960/3678907616.py:17: FutureWarning: The fram

In [2]:
fileName = '../../1000_population.tsv'


lines = []
categories = []


with open(fileName, 'r') as f:
    text = f.read()

lines = text.split('\n')
for l in lines:
    
    categories.append(l.split('\t')[-1])
allcategories=sorted(list(set(categories)))


In [3]:
#For each race of one csv file, average every 1000 P and put it in the csv file
def getAveragePer1000(filepath,n,filename):
    csv_df= pd.read_csv(filepath, low_memory = False)
    # csv_df = pd.DataFrame(csv_data)
    for col in allcategories:
        averages = [np.mean(csv_df.loc[i:i+n-1, col]) for i in range(0, csv_df.shape[0], n)]
        np.savetxt('/data2/wangxuedong/mhc_test_data/first_review/similar_region_race_average_bin_1000/'+col+"_"+filename+'.csv', averages, delimiter = ',')


In [4]:
#run every csvfile in the 'race' folder
for curDir, dirs, files in os.walk(top="/data2/wangxuedong/mhc_test_data/first_review/similar_region_race_integration/"):
    for file in files:
        filename=file.split(".")[0]
        getAveragePer1000(filepath=curDir+file,n=1000,filename=file.split(".")[0])

In [5]:
#for caculating distribution for different race
arrforDistribution_different_race=dict()
for category in allcategories:
    arrforDistribution_different_race[category]=[]

for curDir, dirs, files in os.walk(top="/data2/wangxuedong/mhc_test_data/first_review/similar_region_race_average_bin_1000/"):
    for file in files:
        for category in allcategories:
            if file.startswith(category):
                csv_data=np.loadtxt(open(curDir+file,"rb"),delimiter=",",skiprows=0)
                arrforDistribution_different_race[category]+=csv_data.tolist()
                
        

In [6]:
mean_dict_different_race=dict()
std_dict_different_race=dict()
for category in allcategories:
    mean_dict_different_race[category]=0
    std_dict_different_race[category]=0

for category in allcategories:
    mean_dict_different_race[category]=np.mean(arrforDistribution_different_race[category])
    std_dict_different_race[category]=np.std(arrforDistribution_different_race[category])


In [7]:
mean_dict_different_race#mean for every race

{'ACB': 0.043905219284227484,
 'ASW': 0.061747415774036245,
 'BEB': 0.042622549524204016,
 'CDX': 0.05629086218276262,
 'CEU': 0.03350899999290784,
 'CHB': 0.05201986349516006,
 'CHS': 0.03643899713354041,
 'CLM': 0.04224577966848677,
 'ESN': 0.03623741211034457,
 'FIN': 0.053129961026517944,
 'GBR': 0.05656052651694239,
 'GIH': 0.05157971151605565,
 'GWD': 0.03170015029229307,
 'IBS': 0.0371240314050134,
 'ITU': 0.050724822118831774,
 'JPT': 0.0517456479295656,
 'KHV': 0.04564560273080832,
 'LWK': 0.04973674801295715,
 'MSL': 0.04960771674064523,
 'MXL': 0.05377595974774834,
 'PEL': 0.045806236523102895,
 'PJL': 0.040244478916681585,
 'PUR': 0.04003893507415216,
 'STU': 0.048497323483452925,
 'TSI': 0.05007660537347943,
 'YRI': 0.031315221824936575}

In [8]:
std_dict_different_race#standard deviation for every race

{'ACB': 0.0035837115206611876,
 'ASW': 0.0044435636072078216,
 'BEB': 0.003260312673626174,
 'CDX': 0.00391015689396564,
 'CEU': 0.002766896630079425,
 'CHB': 0.0037059827211937194,
 'CHS': 0.002921456474220746,
 'CLM': 0.003272057962121683,
 'ESN': 0.0031803272910866064,
 'FIN': 0.0038008600285356704,
 'GBR': 0.003936807447491629,
 'GIH': 0.0037221796184063703,
 'GWD': 0.0029073393803639483,
 'IBS': 0.0029858040112478645,
 'ITU': 0.003597951675686167,
 'JPT': 0.003708599258289701,
 'KHV': 0.00343805410624286,
 'LWK': 0.003848934277722462,
 'MSL': 0.003934844393069718,
 'MXL': 0.003865357254429113,
 'PEL': 0.003392906748737866,
 'PJL': 0.003126523821241254,
 'PUR': 0.0031949284883587385,
 'STU': 0.00353413807778532,
 'TSI': 0.0036716230350532964,
 'YRI': 0.002910364879924007}

In [9]:
#calculate p-value, if p-value>0.95 mark it as 1, otherwise, mark it as 0
def compareWith0_01Pvalue(mean_arr,std_arr,filename,filepath):

    arr=[]
    csv_matrix = np.loadtxt(open(filepath,"rb"),delimiter=",",skiprows=0)
    for i in range(len(csv_matrix)):
        mid_res=stats.norm.cdf(csv_matrix[i], mean_arr, std_arr)

        if mid_res>0.95:
            arr.append(1)
        else:
            arr.append(0)
    np.savetxt("../similar_region_race_compare_with_01pvalue/"+filename+".csv", arr, delimiter = ',')
    return filename

In [10]:
for curDir, dirs, files in os.walk(top="/data2/wangxuedong/mhc_test_data/first_review/similar_region_race_average_bin_1000/"):
    for file in files:   
        for category in allcategories:
            if file.startswith(category):
                filename=file[:file.index(".")]
                filepath=os.path.join(curDir,file)
                #print(filepath)
                compareWith0_01Pvalue(mean_arr=mean_dict_different_race[category],std_arr=std_dict_different_race[category],filename=filename,filepath=filepath)

In [11]:
def generatedfandcsv(columns,dic,filename):

    row_names=list(dic.keys())
    a=[]
    bs=[]

    for x in row_names:
        a.append(x)
        bs.append(dic[x])

    dictforDF=dict()
    dictforDF[columns[0]]=a
    dictforDF[columns[1]]=bs


    df_index=pd.DataFrame(dictforDF)
    df_index.to_csv("../similar_region_race_start_length/"+filename+"_start_lenth.csv")
    return df_index


In [12]:
def checkContinuous1(arr):
    res=dict()
    i=0
    while i<len(arr)-1:
        if arr[i]==1:
            count=1
            for j in range(i+1,len(arr),1):
                if arr[j]==1:
                    count+=1                    
                else:                   
                    break
            res[i]=count
            i+=count
        else:
            i+=1
            continue
    return res
            


In [13]:
def getstartAndLength(filepath,filename):
    csv_matrix = np.loadtxt(open(filepath,"rb"),delimiter=",",skiprows=0)
    res=checkContinuous1(csv_matrix)
    df=generatedfandcsv(columns=["start","length"],dic=res,filename=filename)
    return df

In [14]:
for curDir, dirs, files in os.walk(top="../similar_region_race_compare_with_01pvalue/"):
    for file in files:

        filename=file[:file.index(".")]
        filepath=os.path.join(curDir,file)
        getstartAndLength(filepath=filepath,filename=filename)

In [15]:

def countNumofLength(dic,filepath):
    csv_data = pd.read_csv(filepath, low_memory = False)
    csv_df = pd.DataFrame(csv_data)
    arr=np.array(csv_df['length'])
    #print(dic)
    for i in range(len(arr)):
        #print("before iteration",dic)
        if arr[i] in dic:
            #print("in iteration not",dic)
            dic[arr[i]]+=1
        else:
            dic[arr[i]]=1
            #print("in iteration",dic)
    #print(dic)
    dic = {k: v for k, v in sorted(dic.items())}

    return dic

In [16]:

race_dic_except_mhc=dict()
for category in allcategories:
    race_dic_except_mhc[category]=dict()

for curDir, dirs, files in os.walk(top="../similar_region_race_start_length/"):
    for category in allcategories:
        mydic=dict()
        for file in files:        
            if file.startswith(category):
                filename=file[:file.index(".")]
                if "mhc" in filename:
                    continue          
                else:                  
                    filepath=os.path.join(curDir,file)
                    mydic=countNumofLength(mydic,filepath=filepath)
        race_dic_except_mhc[category]=mydic

In [17]:
race_dic_except_mhc#{race:{length of continuous '1': count}} for split_chr...

{'ACB': {1: 939,
  2: 195,
  3: 66,
  4: 36,
  5: 9,
  6: 8,
  7: 2,
  8: 2,
  10: 4,
  11: 2,
  23: 1},
 'ASW': {1: 865,
  2: 206,
  3: 60,
  4: 26,
  5: 18,
  6: 6,
  7: 5,
  8: 4,
  9: 2,
  10: 2,
  11: 1,
  12: 4,
  17: 1,
  19: 1},
 'BEB': {1: 804, 2: 229, 3: 59, 4: 22, 5: 7, 6: 3, 7: 4, 8: 4, 9: 1},
 'CDX': {1: 746, 2: 210, 3: 74, 4: 22, 5: 12, 6: 4, 7: 6, 9: 2, 10: 1, 17: 1},
 'CEU': {1: 816,
  2: 195,
  3: 71,
  4: 23,
  5: 10,
  6: 5,
  7: 1,
  9: 1,
  10: 1,
  11: 1,
  12: 1},
 'CHB': {1: 752, 2: 185, 3: 74, 4: 19, 5: 7, 6: 4, 16: 1, 18: 1},
 'CHS': {1: 828, 2: 211, 3: 46, 4: 28, 5: 12, 6: 7, 7: 2, 8: 1, 9: 3},
 'CLM': {1: 867, 2: 218, 3: 68, 4: 21, 5: 11, 6: 7, 7: 3, 8: 4, 9: 1},
 'ESN': {1: 934, 2: 189, 3: 61, 4: 25, 5: 10, 6: 7, 7: 3, 8: 2, 10: 1},
 'FIN': {1: 775,
  2: 212,
  3: 59,
  4: 27,
  5: 7,
  6: 11,
  7: 3,
  8: 1,
  9: 2,
  13: 1,
  17: 1},
 'GBR': {1: 743, 2: 214, 3: 52, 4: 25, 5: 15, 6: 6, 7: 1, 8: 1, 11: 1, 12: 1},
 'GIH': {1: 835, 2: 228, 3: 70, 4: 26, 5: 7,

In [18]:
mhc_final_dict=dict()
for curDir, dirs, files in os.walk(top="../similar_region_race_start_length/"):
    for category in allcategories:
        mhcdic=dict()
        for file in files:   
            if file.startswith(category):
                filename=file[:file.index(".")]
                if "mhc" in filename:
                    filepath=os.path.join(curDir,file)
                    mhcdic=countNumofLength(mhcdic,filepath=filepath)
        mhc_final_dict[category]=mhcdic
                  

In [19]:
mhc_final_dict#{race:{length of continuous '1': count}} for mhc

{'ACB': {1: 4, 4: 1},
 'ASW': {1: 1},
 'BEB': {1: 4, 2: 1, 4: 1},
 'CDX': {1: 3},
 'CEU': {2: 2},
 'CHB': {1: 2, 2: 1, 3: 1},
 'CHS': {1: 4, 2: 1},
 'CLM': {1: 5},
 'ESN': {1: 1, 2: 3},
 'FIN': {1: 3, 2: 1, 3: 2},
 'GBR': {1: 2, 3: 1},
 'GIH': {1: 4},
 'GWD': {1: 6, 2: 3, 3: 1},
 'IBS': {1: 3},
 'ITU': {1: 2, 2: 2},
 'JPT': {1: 2, 2: 1, 4: 1},
 'KHV': {1: 2},
 'LWK': {1: 3, 2: 1},
 'MSL': {1: 2, 2: 1},
 'MXL': {1: 2, 2: 2},
 'PEL': {4: 2},
 'PJL': {1: 3, 2: 1},
 'PUR': {1: 2, 4: 1},
 'STU': {1: 1},
 'TSI': {1: 3, 2: 1},
 'YRI': {1: 2, 3: 1}}

In [20]:
arr1=[]
for _,v in race_dic_except_mhc.items():
    for k2,arr in v.items():
        for i in range(arr):
            arr1.append(k2)
for _,v in mhc_final_dict.items():
    for k2,arr in v.items():
        for i in range(arr):
            arr1.append(k2)



In [21]:
mean_1=np.mean(arr1)
dev1=np.std(arr1)
print("mean",mean_1)
print("dev",dev1)
mid_res=stats.norm.ppf(0.9999999999,mean_1,dev1)
mid_res

mean 1.471546566077003
dev 1.1427323064753188


8.740856313236694

In [22]:
def getAllPositions(filepath):
    pos=[]
    df=pd.read_csv(filepath)
    pos=df["Unnamed: 0"].to_list()    
    return pos

In [23]:
def findPosition(filepath,start,length):
    arr=[]
    allpos=getAllPositions(filepath=filepath)
    arr.append(allpos[start*1000])
    if (start+length)*1000-1<len(allpos):
        end=allpos[(start+length)*1000-1]
    else:
        end=allpos[-1]
    arr.append(end)
    return arr



In [24]:
#for split_chr..., get complementary region
#/data2/wangxuedong/mhc_test_data/first_review/race_start_length/ACB_chr3_start_lenth.csv
def GetPosition(filepath_start_length,threshold,res):

    csv_data = pd.read_csv(filepath_start_length, low_memory = False)
    csv_df = pd.DataFrame(csv_data)
    arr_start=np.array(csv_df['start'])
    arr_length=np.array(csv_df['length'])
    mydic=dict()
    midarr=[]
    
    for i in range(len(arr_start)):
        mydic[arr_start[i]]=arr_length[i]
    
    name=filepath_start_length.split("/")[-1].split(".")[0].split("_")[1]
    name1=filepath_start_length.split("/")[-1].split(".")[0].split("_")[2]
    if name=="chr6":

        if name1!="mhcpart":
            filepath="/data2/wangxuedong/mhc_test_data/first_review/similar_region_race_integration/"+name+"_"+name1+".csv"
    else:
        filepath="/data2/wangxuedong/mhc_test_data/first_review/similar_region_race_integration/"+name+".csv"

    # filename_components=(filepath.split("/")[-1]).split("_start_lenth.csv")[0].split("_")
    # findfoldername="split_"+filename_components[2]
    # findfoldername_filename=filename_components[3]
    # filepath="../"+findfoldername+"/"+findfoldername_filename+".vcf.gz"

    for k,v in mydic.items():
        if v>=threshold:
            print("filepath",filepath)
            midarr.append(findPosition(filepath=filepath,start=k,length=v))
            
        else:
            continue
        
    if midarr!=[]:
        if name=="chr6":
            if name1!="mhcpart":
                res[name+"_"+name1]=midarr
        else:
            res[name]=midarr

    # if midarr!=[]:
    #     res[findfoldername+"/"+findfoldername_filename]=midarr
    return res

In [25]:
all_race_positons=dict()
for category in allcategories:
    all_race_positons[category]=dict()

for curDir, dirs, files in os.walk(top="../similar_region_race_start_length/"):        
    for category in allcategories:
        res=dict()
        for file in files:
            filename=file[:file.index(".")]
            if "mhc" in filename:
                continue
            else:
                filerace=filename.split("_")[0]
                if filerace==category:                   
                    filepath=os.path.join(curDir,file)
                    res=GetPosition(filepath_start_length=filepath,threshold=9,res=res)
                    all_race_positons[category]=res


filepath /data2/wangxuedong/mhc_test_data/first_review/similar_region_race_integration/chr2.csv


filepath /data2/wangxuedong/mhc_test_data/first_review/similar_region_race_integration/chr3.csv
filepath /data2/wangxuedong/mhc_test_data/first_review/similar_region_race_integration/chr5.csv
filepath /data2/wangxuedong/mhc_test_data/first_review/similar_region_race_integration/chr14.csv
filepath /data2/wangxuedong/mhc_test_data/first_review/similar_region_race_integration/chr16.csv
filepath /data2/wangxuedong/mhc_test_data/first_review/similar_region_race_integration/chr18.csv
filepath /data2/wangxuedong/mhc_test_data/first_review/similar_region_race_integration/chr22.csv
filepath /data2/wangxuedong/mhc_test_data/first_review/similar_region_race_integration/chr2.csv
filepath /data2/wangxuedong/mhc_test_data/first_review/similar_region_race_integration/chr2.csv
filepath /data2/wangxuedong/mhc_test_data/first_review/similar_region_race_integration/chr2.csv
filepath /data2/wangxuedong/mhc_test_data/first_review/similar_region_race_integration/chr2.csv
filepath /data2/wangxuedong/mhc_test

In [26]:
all_race_positons

{'ACB': {'chr2': [[92299419, 92523478]],
  'chr3': [[89474731, 90288204]],
  'chr5': [[62144306, 62605777]],
  'chr14': [[105863438, 106259905]],
  'chr16': [[36332499, 46403568]],
  'chr18': [[19764009, 20828994]],
  'chr22': [[22691667, 22898645]]},
 'ASW': {'chr2': [[92299419, 92507872],
   [92633551, 92902449],
   [93680283, 94160185],
   [94892770, 95546698],
   [202715454, 203549066]],
  'chr3': [[93559360, 94116184]],
  'chr8': [[49383975, 49798135], [103467484, 103900956]],
  'chr10': [[72960046, 73409340]],
  'chr14': [[105859287, 106198900]],
  'chr16': [[36260059, 46728534]]},
 'BEB': {'chr15': [[77138940, 77566277]]},
 'CDX': {'chr8': [[70075248, 70483016]],
  'chr11': [[50311095, 50728899]],
  'chr14': [[66379055, 66778338]],
  'chr20': [[21834643, 22233777]]},
 'CEU': {'chr8': [[84442692, 84821607]],
  'chr12': [[34328866, 34590462]],
  'chr14': [[105917600, 106259905]],
  'chr22': [[22661982, 22898645]]},
 'CHB': {'chr6_partC': [[85501992, 86104633]],
  'chr11': [[470868

In [27]:
#for mhc get complementary region
def GetPositionformhc(filepath_start_length,threshold,res):

    csv_data = pd.read_csv(filepath_start_length, low_memory = False)
    csv_df = pd.DataFrame(csv_data)
    arr_start=np.array(csv_df['start'])
    arr_length=np.array(csv_df['length'])
    mydic=dict()
    midarr=[]
    
    for i in range(len(arr_start)):
        mydic[arr_start[i]]=arr_length[i]

    # filepath="/data2/wangxuedong/mhc_test_data/first_review/similar_region_race_integration/chr6_mhcpart.csv"
    filepath='/data2/wangxuedong/mhc_test_data/first_review/similar_region_csvfiles/posmhc.csv'
    for k,v in mydic.items():
        if v>=threshold:
            print("filepath",filepath)
            midarr.append(findPosition(filepath=filepath,start=k,length=v))
            
        else:
            continue

    if midarr!=[]:
        res["mhc"]=midarr
    return res

In [28]:
mhc_race_positons=dict()
for category in allcategories:
    mhc_race_positons[category]=dict()

for curDir, dirs, files in os.walk(top="../similar_region_race_start_length/"):        
    for category in allcategories:
        res=dict()
        for file in files:
            filename=file[:file.index(".")]
            if "mhc" in filename:
                filerace=filename.split("_")[0]
                if filerace==category:                   
                    filepath=os.path.join(curDir,file)
                    print(filepath)
                    res=GetPositionformhc(filepath_start_length=filepath,threshold=9,res=res)
                    mhc_race_positons[category]=res
                else:
                    continue
            else:
                continue


../similar_region_race_start_length/ACB_chr6_mhcpart_start_lenth.csv
../similar_region_race_start_length/ASW_chr6_mhcpart_start_lenth.csv
../similar_region_race_start_length/BEB_chr6_mhcpart_start_lenth.csv
../similar_region_race_start_length/CDX_chr6_mhcpart_start_lenth.csv
../similar_region_race_start_length/CEU_chr6_mhcpart_start_lenth.csv
../similar_region_race_start_length/CHB_chr6_mhcpart_start_lenth.csv
../similar_region_race_start_length/CHS_chr6_mhcpart_start_lenth.csv
../similar_region_race_start_length/CLM_chr6_mhcpart_start_lenth.csv
../similar_region_race_start_length/ESN_chr6_mhcpart_start_lenth.csv
../similar_region_race_start_length/FIN_chr6_mhcpart_start_lenth.csv
../similar_region_race_start_length/GBR_chr6_mhcpart_start_lenth.csv
../similar_region_race_start_length/GIH_chr6_mhcpart_start_lenth.csv
../similar_region_race_start_length/GWD_chr6_mhcpart_start_lenth.csv
../similar_region_race_start_length/IBS_chr6_mhcpart_start_lenth.csv
../similar_region_race_start_lengt

In [29]:
mhc_race_positons

{'ACB': {},
 'ASW': {},
 'BEB': {},
 'CDX': {},
 'CEU': {},
 'CHB': {},
 'CHS': {},
 'CLM': {},
 'ESN': {},
 'FIN': {},
 'GBR': {},
 'GIH': {},
 'GWD': {},
 'IBS': {},
 'ITU': {},
 'JPT': {},
 'KHV': {},
 'LWK': {},
 'MSL': {},
 'MXL': {},
 'PEL': {},
 'PJL': {},
 'PUR': {},
 'STU': {},
 'TSI': {},
 'YRI': {}}